In [4]:
import open3d as o3d
import numpy as np
import pandas as pd
from pdb import set_trace as bp
from PIL import Image

In [53]:
mesh_file = 'frame_000001.obj'
mesh = o3d.io.read_triangle_mesh(mesh_file)

[Open3D WARNING] Read OBJ failed: Material file [ frame_000001.mtl ] not found.
Failed to load material file(s). Use default material.



In [55]:
def normalize_obj(mesh_v):
    dims = [max(mesh_v[:, 0]) - min(mesh_v[:, 0]),
            max(mesh_v[:, 1]) - min(mesh_v[:, 1]),
            max(mesh_v[:, 2]) - min(mesh_v[:, 2])]
    scale = 1.0 / max(dims)
    pivot = np.array([(min(mesh_v[:, 0]) + max(mesh_v[:, 0])) / 2, min(mesh_v[:, 1]),
                      (min(mesh_v[:, 2]) + max(mesh_v[:, 2])) / 2])
    mesh_v[:, 0] -= pivot[0]
    mesh_v[:, 1] -= pivot[1]
    mesh_v[:, 2] -= pivot[2]
    mesh_v *= scale
    return mesh_v, pivot, scale

In [56]:
def drawSphere(center, radius, color=[0.0,0.0,0.0]):
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
    transform_mat = np.eye(4)
    transform_mat[0:3, -1] = center
    mesh_sphere.transform(transform_mat)
    mesh_sphere.paint_uniform_color(color)
    return mesh_sphere


def drawCone(bottom_center, top_position, color=[0.6, 0.6, 0.9]):
    cone = o3d.geometry.TriangleMesh.create_cone(radius=0.007, height=np.linalg.norm(top_position - bottom_center)+1e-6)
    line1 = np.array([0.0, 0.0, 1.0])
    line2 = (top_position - bottom_center) / (np.linalg.norm(top_position - bottom_center)+1e-6)
    v = np.cross(line1, line2)
    c = np.dot(line1, line2) + 1e-8
    k = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    R = np.eye(3) + k + np.matmul(k, k) * (1 / (1 + c))
    if np.abs(c + 1.0) < 1e-4: # the above formula doesn't apply when cos(∠(𝑎,𝑏))=−1
        R = np.array([[-1, 0, 0], [0, 1, 0], [0, 0, -1]])
    T = bottom_center + 5e-3 * line2
    #print(R)
    cone.transform(np.concatenate((np.concatenate((R, T[:, np.newaxis]), axis=1), np.array([[0.0, 0.0, 0.0, 1.0]])), axis=0))
    cone.paint_uniform_color(color)
    return cone

In [57]:
info = np.load('african_buffalo_female.npy',allow_pickle=True)

In [60]:
info = info.item()

In [61]:
def change(pos_array):
    p = pos_array
    return (p[0],p[2],-p[1]+0.2)

In [63]:
vis = o3d.visualization.Visualizer()
vis.create_window()
mesh_ls = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
mesh_ls.colors = o3d.utility.Vector3dVector([[0.8, 0.8, 0.8] for i in range(len(mesh_ls.lines))])
vis.add_geometry(mesh_ls)
this_level = [info['def_c_root_joint']]
vis.add_geometry(drawSphere(this_level[0]['head'], 0.008, color=[1.0, 0.0, 0.0]))
while this_level:
    next_level = []
    for p_node in this_level:
#         bp()
#         print(p_node['head'])
        vis.add_geometry(drawSphere(change(p_node['head']), 0.008, color=[1.0, 0.0, 0.0])) # [0.3, 0.1, 0.1]
        vis.add_geometry(drawSphere(change(p_node['tail']), 0.008, color=[1.0, 0.0, 0.0]))
        vis.add_geometry(drawCone(np.array(change(p_node['head'])), np.array(change(p_node['tail']))))
        if p_node['is_leaf']==0:
            next_level+=[info[info_index] for info_index in  p_node['children']]
        else:
            continue
    this_level = next_level
vis.run()
image1 = vis.capture_screen_float_buffer()
vis.destroy_window()